In [1]:
import sys
sys.path.append("./")

import numpy as np
from core import Arcface_model,config
from evaluate.evaluate import evaluation
import pickle
import io
from scipy import misc
import tensorflow as tf


if __name__ == "__main__":

    img_size=112
    batch_size=256
    model_path="./model/Arcface-300000"
    f = open("./eval_record.txt", 'w')
    
    image=tf.placeholder(tf.float32,[batch_size,img_size,img_size,3],name='image')
    train_phase_dropout = tf.placeholder(dtype=tf.bool, shape=None, name='train_phase_dropout')
    train_phase_bn = tf.placeholder(dtype=tf.bool, shape=None, name='train_phase_bn')    
    
    embd, _ = Arcface_model.get_embd(image, train_phase_dropout, train_phase_bn, config.model_params)
    saver=tf.train.Saver()

    tf_config = tf.ConfigProto(allow_soft_placement=True)  
    tf_config.gpu_options.allow_growth = True 
    with tf.Session(config = tf_config) as sess:
        saver.restore(sess,model_path)
        for dataset_path in config.eval_datasets:
            tpr, fpr, accuracy, best_thresholds = evaluation(sess, batch_size, img_size, dataset_path, dropout_flag=config.eval_dropout_flag, bn_flag=config.eval_bn_flag, embd=embd, image=image, train_phase_dropout=train_phase_dropout, train_phase_bn=train_phase_bn) 
            print("%s datasets get %.3f acc,best_thresholds are: "%(dataset_path.split("/")[-1].split(".")[0],accuracy), best_thresholds)
            f.write("%s ,\t accuracy: %.3f , \t best_thresholds are: "%(dataset_path.split("/")[-1].split(".")[0],accuracy), " ".join(best_thresholds)+'\n')
    f.close() 

/home/dell/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Restoring parameters from ./model/Arcface-300000
reading ./data/lfw.bin
done!
lfw datasets get 0.950 acc,best_thresholds are:  [1.49 1.49 1.49 1.49 1.49 1.49 1.49 1.49 1.49 1.49]


TypeError: sequence item 0: expected str instance, numpy.float64 found